In [1]:
import subprocess
import os

# Define paths
fastq_data = "/home/abozar/pathogenereads/dashtnaz/paired_reads.fastq"
reference_genome = "/home/abozar/pathogenereads/blast/fetched_sequences.fasta"
minimap2_path = "/usr/bin/minimap2"
outputs_dir = "/home/abozar/pathogenereads/blast"

# Define output filename for the alignment file (modify as needed)
sam_file = os.path.join(outputs_dir, "aligned.sam")

# Build minimap2 command
minimap2_command = [
    minimap2_path,
    "-ax", "sr",  # Paired-end reads alignment mode
    reference_genome,
    fastq_data,
    "-o", sam_file,  # Output SAM file
]

# Run minimap2 alignment
try:
    subprocess.run(minimap2_command, check=True)
except subprocess.CalledProcessError as e:
    print("Minimap2 alignment failed:", e)
    exit(1)

# Check if the output file is empty
if os.path.getsize(sam_file) == 0:
    print("Error: Aligned SAM file is empty.")
    exit(1)

# Print confirmation message
print("Alignment results saved to:", sam_file)


[M::mm_idx_gen::0.001*1.66] collected minimizers
[M::mm_idx_gen::0.002*2.09] sorted minimizers
[M::main::0.002*2.07] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.002*2.07] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.002*1.98] distinct minimizers: 3186 (100.00% are singletons); average occurrences: 1.000; average spacing: 6.056; total length: 19296
[M::worker_pipeline::1.242*1.00] mapped 338626 sequences
[M::worker_pipeline::1.971*1.20] mapped 338602 sequences
[M::worker_pipeline::2.640*1.30] mapped 338590 sequences
[M::worker_pipeline::3.447*1.33] mapped 338552 sequences
[M::worker_pipeline::4.093*1.38] mapped 338352 sequences
[M::worker_pipeline::4.765*1.41] mapped 338488 sequences
[M::worker_pipeline::5.463*1.42] mapped 338542 sequences
[M::worker_pipeline::6.326*1.42] mapped 338532 sequences
[M::worker_pipeline::7.016*1.43] mapped 338520 sequences
[M::worker_pipeline::7.727*1.45] mapped 338632 sequences
[M

Alignment results saved to: /home/abozar/pathogenereads/blast/aligned.sam


[M::worker_pipeline::162.051*0.84] mapped 339090 sequences
[M::worker_pipeline::162.112*0.84] mapped 131352 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: /usr/bin/minimap2 -ax sr -o /home/abozar/pathogenereads/blast/aligned.sam /home/abozar/pathogenereads/blast/fetched_sequences.fasta /home/abozar/pathogenereads/dashtnaz/paired_reads.fastq
[M::main] Real time: 162.117 sec; CPU: 136.590 sec; Peak RSS: 0.438 GB


In [2]:
from collections import defaultdict

# Function to parse SAM file and extract aligned sequences
def parse_sam(sam_file):
    aligned_sequences = defaultdict(list)
    with open(sam_file, 'r') as f:
        for line in f:
            if not line.startswith('@'):
                fields = line.split('\t')
                read_id = fields[0]
                flag = int(fields[1])
                ref_name = fields[2]
                start_pos = int(fields[3])
                cigar = fields[5]
                seq = fields[9]
                if flag & 4 == 0:  # Check if read is mapped
                    aligned_sequences[ref_name].append((start_pos, cigar, seq))
    return aligned_sequences

# Function to generate consensus sequence
def generate_consensus(aligned_sequences):
    consensus = {}
    for ref_name, alignments in aligned_sequences.items():
        ref_length = max(end_pos + len(seq) for end_pos, _, seq in alignments)
        ref_seq = ['N'] * ref_length
        for start_pos, cigar, seq in alignments:
            ref_pos = start_pos - 1
            for op in parse_cigar(cigar):
                if op[0] == 'M':  # Match
                    ref_seq[ref_pos:ref_pos+op[1]] = list(seq)[:op[1]]
                    ref_pos += op[1]
                elif op[0] == 'D':  # Deletion
                    ref_pos += op[1]
                elif op[0] == 'I':  # Insertion
                    ref_seq[ref_pos:ref_pos] = ['N'] * op[1]
                elif op[0] == 'S':  # Soft clipping
                    ref_pos += op[1]
        consensus[ref_name] = ''.join(ref_seq)
    return consensus

# Function to parse CIGAR string
def parse_cigar(cigar):
    operations = []
    count = ''
    for char in cigar:
        if char.isdigit():
            count += char
        else:
            operations.append((char, int(count)))
            count = ''
    return operations

# Main code
sam_file = "/home/abozar/pathogenereads/blast/aligned.sam"
aligned_sequences = parse_sam(sam_file)
consensus = generate_consensus(aligned_sequences)

# Print or save consensus sequences
for ref_name, seq in consensus.items():
    print(f">Consensus_{ref_name}")
    print(seq)


>Consensus_NC_001661.1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN